In [41]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

CALCULO DEL PRIMER KPI:

Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

In [34]:

archivo_csv = 'df_homicidios_cleaned.csv'
df_homicidios_cleaned = pd.read_csv(archivo_csv)

# CreaNDO una copia del DataFrame
df_copia = df_homicidios_cleaned.copy()



In [35]:
# Especificabdo ruta del archivo
ruta_archivo = 'Poblacion_totalCABA.xlsx'

# Cargando el archivo XLSX como un DataFrame, saltando las primeras 1 fila (para omitir la primera fila con nombres de columnas)
df_poblacion = pd.read_excel(ruta_archivo, skiprows=1)

# Muestra las primeras filas del DataFrame para verificar que se haya cargado correctamente
df_poblacion


,Año,Total
0,2016,3059122.0
1,2017,3063728.0
2,2018,3068043.0
3,2019,3072029.0
4,2020,3075646.0
5,2021,3078836.0


Con la columna SEMESTRE se realiza la agrupación por AÑO y SEMESTRE para poder sumar la cantidad de MUERTES que corresponde a cada uno

In [36]:
agrupacion_semestre = (df_copia.groupby(["AÑO", "SEMESTRE"]).size().reset_index(name="MUERTES"))
agrupacion_semestre

,AÑO,SEMESTRE,MUERTES
0,2016,1,65
1,2016,2,81
2,2017,1,69
3,2017,2,71
4,2018,1,70
5,2018,2,79
6,2019,1,57
7,2019,2,46
8,2020,1,31
9,2020,2,50


Se calcula una nueva columna llamada SEMESTRE ANTERIOR donde se calcula para cada semestre pasado la cantidad de decesos:

In [37]:
agrupacion_semestre["SEMESTRE_ANTERIOR"] = agrupacion_semestre["MUERTES"].shift(periods=1, fill_value=0)
agrupacion_semestre

,AÑO,SEMESTRE,MUERTES,SEMESTRE_ANTERIOR
0,2016,1,65,0
1,2016,2,81,65
2,2017,1,69,81
3,2017,2,71,69
4,2018,1,70,71
5,2018,2,79,70
6,2019,1,57,79
7,2019,2,46,57
8,2020,1,31,46
9,2020,2,50,31


Se calcula la tasa de homicidios para cada semestre cada 100000 habitantes


In [38]:
# Seleccionas la población total del año 2021
pob_total_2021 = df_poblacion[df_poblacion["Año"] == 2021]["Total"].iloc[0]
pob_total_2021

3078836.0

In [39]:
# tasa de homicidios por semestre
agrupacion_semestre["TASA_MUERTES"] = (agrupacion_semestre["MUERTES"] / pob_total_2021) * 100000

# Se calcula la tasa de homicidios del semestre inmediatamente anterior
agrupacion_semestre["TASA_SEMESTRE_ANTERIOR"] = (
    agrupacion_semestre["SEMESTRE_ANTERIOR"] / pob_total_2021
) * 100000
# Vemos las tasas de homicidios
agrupacion_semestre.head()

,AÑO,SEMESTRE,MUERTES,SEMESTRE_ANTERIOR,TASA_MUERTES,TASA_SEMESTRE_ANTERIOR
0,2016,1,65,0,2.111187,0.000000
1,2016,2,81,65,2.630864,2.111187
2,2017,1,69,81,2.241107,2.630864
3,2017,2,71,69,2.306066,2.241107
4,2018,1,70,71,2.273587,2.306066


Se crea una columna con la diferencia porcentual semestre a semestre


In [40]:
agrupacion_semestre["dif_porcentual"] = ((agrupacion_semestre["TASA_MUERTES"]
        - agrupacion_semestre["TASA_SEMESTRE_ANTERIOR"] )/ agrupacion_semestre["TASA_SEMESTRE_ANTERIOR"] * 100)
agrupacion_semestre

,AÑO,SEMESTRE,MUERTES,SEMESTRE_ANTERIOR,TASA_MUERTES,TASA_SEMESTRE_ANTERIOR,dif_porcentual
0,2016,1,65,0,2.111187,0.000000,inf
1,2016,2,81,65,2.630864,2.111187,24.615385
2,2017,1,69,81,2.241107,2.630864,-14.814815
3,2017,2,71,69,2.306066,2.241107,2.898551
4,2018,1,70,71,2.273587,2.306066,-1.408451
5,2018,2,79,70,2.565905,2.273587,12.857143
6,2019,1,57,79,1.851349,2.565905,-27.848101
7,2019,2,46,57,1.494071,1.851349,-19.298246
8,2020,1,31,46,1.006874,1.494071,-32.608696
9,2020,2,50,31,1.623990,1.006874,61.290323


Podemos observar varias tendencias y cambios en la tasa de homicidios en siniestros viales a lo largo de los semestres:

A lo largo de los semestres, se observa una tendencia general a la baja en la tasa de homicidios en siniestros viales, aunque hay semestres con aumentos respecto al semestre anterior. El semestre 1 del 2020 muestra la mayor baja, probablemente debido a la menor circulación de autos por las restricciones por la pandemia de COVID19

El último semestre analizado (segundo semestre de 2021), se logró una reducción del 23% en la tasa de muertes en comparación con el semestre anterior, superando el objetivo del KPI de una reducción del 10%. Esto indica que las medidas implementadas durante este período fueron efectivas.

Estos datos subrayan la importancia de seguir implementando y mejorando las medidas de seguridad vial para reducir la tasa de muertes en siniestros viales. 

In [50]:
# Guardar como archivo CSV
agrupacion_semestre.to_csv('agrupacion_semestre.csv', index=False)


SEGUNDO KPI:

Reducir en un 7% la cantidad de accidentes mortales de motociclistas en el último año, en CABA, respecto al año anterior.


Se filtra y selecciona registros donde la victima es motociclista


In [20]:
victima_moto = df_copia[(df_copia["VICTIMA"] == "MOTO")]

Se agrupa por año y se realiza el conteo de MUERTES


In [21]:
victima_moto = victima_moto.groupby(["AÑO"]).size().reset_index(name="MUERTES")
victima_moto

,AÑO,MUERTES
0,2016,65
1,2017,56
2,2018,57
3,2019,49
4,2020,28
5,2021,46


Se crea una nueva variable AÑO_ANTERIOR donde se obtiene la cantidad de MUERTES ocurridos el año anterior

In [22]:
victima_moto["AÑO_ANTERIOR"] = victima_moto["MUERTES"].shift(periods=1, fill_value=0)
victima_moto

,AÑO,MUERTES,AÑO_ANTERIOR
0,2016,65,0
1,2017,56,65
2,2018,57,56
3,2019,49,57
4,2020,28,49
5,2021,46,28


Se calcula la tasa de MUERTES cada 100000 habitantes teniendo en cuenta que la población de Cuidad Autónoma de Buenos Aires al censo 2021


In [23]:
#  tasa de muertes cada 100000 habitantes para el año actual
victima_moto["TASA_MUERTES"] = (victima_moto["MUERTES"] / pob_total_2021) * 100000

# tasa de muertes cada 100000 habitantes para el año anterior 
victima_moto["TASA_AÑO_ANTERIOR"] = (victima_moto["AÑO_ANTERIOR"] / pob_total_2021) * 100000
victima_moto

,AÑO,MUERTES,AÑO_ANTERIOR,TASA_MUERTES,TASA_AÑO_ANTERIOR
0,2016,65,0,2.111187,0.000000
1,2017,56,65,1.818869,2.111187
2,2018,57,56,1.851349,1.818869
3,2019,49,57,1.591511,1.851349
4,2020,28,49,0.909435,1.591511
5,2021,46,28,1.494071,0.909435


Se obtiene la Variación Porcentual de las MUERTES año a año


In [24]:
victima_moto["variacion"] = ((victima_moto["TASA_MUERTES"] - victima_moto["TASA_AÑO_ANTERIOR"]) / victima_moto["TASA_AÑO_ANTERIOR"]) * 100

victima_moto["variacion"] = victima_moto["variacion"].replace([np.inf, -np.inf], np.nan)

victima_moto

,AÑO,MUERTES,AÑO_ANTERIOR,TASA_MUERTES,TASA_AÑO_ANTERIOR,variacion
0,2016,65,0,2.111187,0.000000,NaN
1,2017,56,65,1.818869,2.111187,-13.846154
2,2018,57,56,1.851349,1.818869,1.785714
3,2019,49,57,1.591511,1.851349,-14.035088
4,2020,28,49,0.909435,1.591511,-42.857143
5,2021,46,28,1.494071,0.909435,64.285714


Podemos observar, que la cantidades de accidentes mortales de motociclistas aumentaron de 28 en 2020 a 46 en 2021, lo que quiere decir que no se logró la reducción del 7% en accidentes mortales de motociclistas. De hecho, hubo un aumento del 64.29% en accidentes mortales de motociclistas de 2020 a 2021.

A partir de los datos analizados, podemos observar varias tendencias y cambios en el número de accidentes mortales de motociclistas a lo largo de los años:

En 2017, los accidentes mortales de motociclistas disminuyeron en un 13.85% en comparación con el año anterior. Esto podría ser un indicativo de que las estrategias de seguridad vial implementadas en ese año tuvieron un impacto positivo.

En 2018, hubo un aumento del 1.79% en comparación con 2017. Aunque es un aumento, la tasa de cambio es relativamente pequeña, lo que podría indicar que la situación se mantuvo relativamente estable.

En 2019, hubo una disminución del 14.03% en comparación con 2018. Esto es una buena señal y podría indicar que las medidas de seguridad implementadas ese año fueron efectivas.

En 2020, hubo una disminución significativa del 43% en comparación con 2019. Este es un cambio notable y podría estar relacionado con factores excepcionales como la pandemia de COVID-19, que pudo haber reducido la cantidad de tráfico y, por lo tanto, el número de accidentes.

En 2021, hubo un aumento significativo del 64.29% en comparación con 2020. Este es un cambio preocupante y sugiere que se deben tomar medidas para abordar este aumento.

Estes análisis subraya la importancia de seguir implementando y mejorando las medidas de seguridad para los motociclistas. También destacan cómo factores externos, como una pandemia, pueden tener un impacto significativo en las tasas de accidentes.

In [51]:
# Guardar como archivo CSV
victima_moto.to_csv('victima_moto.csv', index=False)


TERCER KPI:

Reducir en un 10% la cantidad de accidentes mortales de PEATONES en el último año, en CABA, respecto al año anterior.

Se filtra y selecciona registros donde la victima es PEATON:


In [28]:
victima_peaton = df_copia[(df_copia["VICTIMA"] == "PEATON")]

Se agrupa por año y se realiza el conteo de MUERTES


In [29]:
victima_peaton = victima_peaton.groupby(["AÑO"]).size().reset_index(name="MUERTES")
victima_peaton

,AÑO,MUERTES
0,2016,49
1,2017,49
2,2018,66
3,2019,35
4,2020,34
5,2021,33


Se crea una nueva variable AÑO_ANTERIOR donde se obtiene la cantidad de MUERTES ocurridos el año anterior

In [30]:
victima_peaton["AÑO_ANTERIOR"] = victima_peaton["MUERTES"].shift(periods=1, fill_value=0)
victima_peaton

,AÑO,MUERTES,AÑO_ANTERIOR
0,2016,49,0
1,2017,49,49
2,2018,66,49
3,2019,35,66
4,2020,34,35
5,2021,33,34


Se calcula la tasa de MUERTES cada 100000 habitantes teniendo en cuenta que la población de Cuidad Autónoma de Buenos Aires al censo 2021


In [31]:
#  tasa de muertes cada 100000 habitantes para el año actual
victima_peaton["TASA_MUERTES"] = (victima_peaton["MUERTES"] / pob_total_2021) * 100000

# tasa de muertes cada 100000 habitantes para el año anterior 
victima_peaton["TASA_AÑO_ANTERIOR"] = (victima_peaton["AÑO_ANTERIOR"] / pob_total_2021) * 100000
victima_peaton

,AÑO,MUERTES,AÑO_ANTERIOR,TASA_MUERTES,TASA_AÑO_ANTERIOR
0,2016,49,0,1.591511,0.000000
1,2017,49,49,1.591511,1.591511
2,2018,66,49,2.143667,1.591511
3,2019,35,66,1.136793,2.143667
4,2020,34,35,1.104313,1.136793
5,2021,33,34,1.071834,1.104313


Se obtiene la Variación Porcentual de las MUERTES año a año


In [32]:
victima_peaton["variacion"] = ((victima_peaton["TASA_MUERTES"] - victima_peaton["TASA_AÑO_ANTERIOR"]) / victima_peaton["TASA_AÑO_ANTERIOR"]) * 100

victima_peaton["variacion"] = victima_peaton["variacion"].replace([np.inf, -np.inf], np.nan)

victima_peaton

,AÑO,MUERTES,AÑO_ANTERIOR,TASA_MUERTES,TASA_AÑO_ANTERIOR,variacion
0,2016,49,0,1.591511,0.000000,NaN
1,2017,49,49,1.591511,1.591511,0.000000
2,2018,66,49,2.143667,1.591511,34.693878
3,2019,35,66,1.136793,2.143667,-46.969697
4,2020,34,35,1.104313,1.136793,-2.857143
5,2021,33,34,1.071834,1.104313,-2.941176


El último valor en la columna variacion es -2.94%, significa que la tasa de muertes de peatones en el último año (2021) ha disminuido en aproximadamente un 2.94% en comparación con el año anterior (2020), entonces -2.94% no cumple con el KPI, ya que la reducción es menor al 10%.

In [52]:
# Guardar como archivo CSV
victima_peaton.to_csv('victima_peaton.csv', index=False)